In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

In [2]:
weather_data = ['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']
years = ["2016","2017"]
weather = ['windspeedKmph','winddirDegree','weatherCode','precipMM','visibility','pressure','cloudcover','DewPointF','WindGustKmph','tempF','WindChillF','humidity','date','time', 'airport']

weatherdf = pd.DataFrame()

for airport in weather_data:
    for year in years:
        for i in range(1,13):
            with open("weather/"+airport+"/"+year+"-"+str(i)+".json", 'r') as f:
                data = json.load(f)
            a = json_normalize(data['data']['weather'], 'hourly', ['date'] )
            b = json_normalize(data['data']['request'])
            a['airport'] = str(b['query'])[5:8]
            a = a.filter(weather)
            weatherdf = pd.concat([weatherdf,a])


In [3]:
flightdf = pd.DataFrame()

for year in years:
    for i in range(1,13):
        df = pd.read_csv('Flight-Data/'+year+'_original/'+year+'/On_Time_On_Time_Performance_'+year+'_'+str(i)+'/On_Time_On_Time_Performance_'+year+'_'+str(i)+'.csv', engine = 'python')
        flight = ['FlightDate','Quarter','Year','Month','DayofMonth','DepTime','DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','DestAirportID','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes','Origin','Dest']
        df = df.filter(flight)
        
        df = df[df['Origin'].isin(weather_data)]
        df = df[df['Dest'].isin(weather_data)]
        
        flightdf = pd.concat([flightdf,df])
        

In [4]:
def alterdeptime(r):
    if(r['CRSDepTime']%100>30):
        return ((r['CRSDepTime']//100 + 1)*100) 
    else:
        return ((r['CRSDepTime']//100)*100)
    
def alterarrtime(r):
    if(r['CRSArrTime']%100>30):
        return ((r['CRSArrTime']//100 + 1)*100) 
    else:
        return ((r['CRSArrTime']//100)*100)

flightdf["newarrtime"] = flightdf.apply(lambda r: alterarrtime(r), axis = 1)
flightdf["newdepttime"] = flightdf.apply(lambda r: alterdeptime(r), axis = 1)


In [5]:
def unique(r):
  return r["airport"]+"||"+r["date"]+"||"+str(r["time"])

weatherdf["key"] = weatherdf.apply(lambda r: unique(r), axis = 1)


In [6]:
weatherdf

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport,key
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,2016-01-01,0,ATL,ATL||2016-01-01||0
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,2016-01-01,100,ATL,ATL||2016-01-01||100
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,2016-01-01,200,ATL,ATL||2016-01-01||200
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,300,ATL,ATL||2016-01-01||300
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,400,ATL,ATL||2016-01-01||400
5,17,315,122,0.0,10,1024,100,38,29,40,33,93,2016-01-01,500,ATL,ATL||2016-01-01||500
6,18,315,122,0.0,10,1024,100,38,28,40,32,93,2016-01-01,600,ATL,ATL||2016-01-01||600
7,18,317,122,0.0,10,1024,100,38,26,41,33,93,2016-01-01,700,ATL,ATL||2016-01-01||700
8,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,800,ATL,ATL||2016-01-01||800
9,17,319,122,0.0,10,1025,100,39,20,42,35,92,2016-01-01,900,ATL,ATL||2016-01-01||900


In [7]:
flightdf

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin,Dest,newarrtime,newdepttime
57,2016-01-01,1,2016,1,1,741.0,0.0,745,0.0,14747,12478,1610.0,1602,0.0,8.0,SEA,JFK,1600,800
58,2016-01-02,1,2016,1,2,737.0,0.0,745,0.0,14747,12478,1613.0,1602,0.0,11.0,SEA,JFK,1600,800
59,2016-01-03,1,2016,1,3,743.0,0.0,745,0.0,14747,12478,1547.0,1602,0.0,0.0,SEA,JFK,1600,800
60,2016-01-04,1,2016,1,4,737.0,0.0,745,0.0,14747,12478,1551.0,1602,0.0,0.0,SEA,JFK,1600,800
61,2016-01-05,1,2016,1,5,708.0,0.0,710,0.0,14747,12478,1524.0,1527,0.0,0.0,SEA,JFK,1500,700
62,2016-01-06,1,2016,1,6,703.0,0.0,710,0.0,14747,12478,1518.0,1527,0.0,0.0,SEA,JFK,1500,700
63,2016-01-07,1,2016,1,7,704.0,0.0,710,0.0,14747,12478,1522.0,1527,0.0,0.0,SEA,JFK,1500,700
64,2016-01-08,1,2016,1,8,701.0,0.0,710,0.0,14747,12478,1527.0,1527,0.0,0.0,SEA,JFK,1500,700
65,2016-01-09,1,2016,1,9,703.0,0.0,710,0.0,14747,12478,1536.0,1527,0.0,9.0,SEA,JFK,1500,700
66,2016-01-10,1,2016,1,10,711.0,0.0,710,1.0,14747,12478,1611.0,1527,1.0,44.0,SEA,JFK,1500,700


In [8]:
def unique(r):
  return r["Dest"]+"||"+r["FlightDate"]+"||"+str(r["newarrtime"])

flightdf["key"] = flightdf.apply(lambda r: unique(r), axis = 1)

final = pd.merge(flightdf, weatherdf, on='key', how='inner')

In [9]:
def unique(r):
  return r["Origin"]+"||"+r["FlightDate"]+"||"+str(r["newdepttime"])

flightdf["key"] = flightdf.apply(lambda r: unique(r), axis = 1)

final = pd.merge(final, weatherdf, on='key', how='inner',suffixes=('_arr','_dept'))

In [10]:
final

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,...,pressure_dept,cloudcover_dept,DewPointF_dept,WindGustKmph_dept,tempF_dept,WindChillF_dept,humidity_dept,date_dept,time_dept,airport_dept
0,2016-01-01,1,2016,1,1,741.0,0.0,745,0.0,14747,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK
1,2016-01-01,1,2016,1,1,1306.0,0.0,1310,0.0,13303,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK
2,2016-01-01,1,2016,1,1,1408.0,1.0,1330,38.0,13204,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK
3,2016-01-01,1,2016,1,1,758.0,0.0,800,0.0,12892,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK
4,2016-01-01,1,2016,1,1,1236.0,0.0,1240,0.0,13930,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK
5,2016-01-01,1,2016,1,1,1144.0,0.0,1150,0.0,11298,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK
6,2016-01-01,1,2016,1,1,1437.0,0.0,1424,13.0,11057,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK
7,2016-01-01,1,2016,1,1,729.0,0.0,730,0.0,12892,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK
8,2016-01-01,1,2016,1,1,1325.0,0.0,1326,0.0,13204,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK
9,2016-01-01,1,2016,1,1,736.0,0.0,735,1.0,12892,...,1015,0,32,26,41,32,74,2016-01-01,1600,JFK


In [15]:

del final["key"]
del final["OriginAirportID"]
del final['DestAirportID']
del final['newarrtime']
del final['newdepttime']
del final['FlightDate']

final.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1827267 entries, 0 to 1827266
Data columns (total 44 columns):
Quarter               int64
Year                  int64
Month                 int64
DayofMonth            int64
DepTime               float64
DepDel15              float64
CRSDepTime            int64
DepDelayMinutes       float64
ArrTime               float64
CRSArrTime            int64
ArrDel15              float64
ArrDelayMinutes       float64
Origin                object
Dest                  object
windspeedKmph_arr     object
winddirDegree_arr     object
weatherCode_arr       object
precipMM_arr          object
visibility_arr        object
pressure_arr          object
cloudcover_arr        object
DewPointF_arr         object
WindGustKmph_arr      object
tempF_arr             object
WindChillF_arr        object
humidity_arr          object
date_arr              object
time_arr              object
airport_arr           object
windspeedKmph_dept    object
winddirDegree_dep

In [16]:
final.isna().any()

Quarter               False
Year                  False
Month                 False
DayofMonth            False
DepTime                True
DepDel15               True
CRSDepTime            False
DepDelayMinutes        True
ArrTime                True
CRSArrTime            False
ArrDel15               True
ArrDelayMinutes        True
Origin                False
Dest                  False
windspeedKmph_arr     False
winddirDegree_arr     False
weatherCode_arr       False
precipMM_arr          False
visibility_arr        False
pressure_arr          False
cloudcover_arr        False
DewPointF_arr         False
WindGustKmph_arr      False
tempF_arr             False
WindChillF_arr        False
humidity_arr          False
date_arr              False
time_arr              False
airport_arr           False
windspeedKmph_dept    False
winddirDegree_dept    False
weatherCode_dept      False
precipMM_dept         False
visibility_dept       False
pressure_dept         False
cloudcover_dept     

In [17]:
final = final.dropna()

final.to_csv('/home/rishi/ML project/final.csv')